# 实验 3.5 - 学员笔记本

## 概述

本实验是模块 3 引导式实验的延续。

在本实验中，您将部署经过训练的模型并根据该模型进行预测。然后，您将删除终端节点，并对测试数据集执行批量转换。


## 业务场景简介

您在一家医疗保健服务提供商工作，并希望改善骨科患者的异常检测。

您的任务是利用机器学习 (ML) 解决此问题。您可以使用包含六个生物力学特征且目标为*正常*或*异常*的数据集。您可以使用此数据集训练 ML 模型，以预测患者是否会出现异常。


## 关于该数据集

该生物医学数据集由 Henrique da Mota 博士在法国里昂 Médico-Chirurgical de Réadaptation des Massues 中心的整形外科应用研究组 (GARO) 实习期间创建。这些数据分到两个不同但相关的分类任务中。

第一项任务是将患者归类为以下三类之一： 

- *正常*（100 名患者）
- *椎间盘疝*（60 名患者）
- *脊椎滑脱*（150 名患者）

对于第二个任务，则是将*椎间盘疝*和*脊椎滑脱*合并为一个类别，标记为*异常*。因此，在第二个任务中，患者属于以下两个类别之一：*正常*（100 名患者）或*异常*（210 名患者）。


## 属性信息

数据集中的每名患者都由六个生物力学属性表示，这些属性（顺序如下）是根据骨盆和腰椎的形状和方向得出的： 

- 骨盆入射角
- 骨盆倾斜角
- 腰椎前凸角
- 骶骨倾斜角
- 骨盆半径
- 脊椎滑脱等级

以下约定用于分类标签： 
- 椎间盘疝 (DH)
- 脊椎滑脱 (SL)
- 正常 (NO) 
- 异常 (AB)

有关此数据集的更多信息，请参阅[脊柱数据集网页](http://archive.ics.uci.edu/ml/datasets/Vertebral+Column)。


## 数据集属性

该数据集来自：
Dua, D. 和 Graff, C.（2019 年）。UCI 机器学习存储库 (http://archive.ics.uci.edu/ml)。加州尔湾市：加利福尼亚大学信息与计算机科学学院。


# 实验设置

由于此解决方案分散在模块中的多个实验中，因此您需要执行以下单元格中的内容，以便加载数据并训练要部署的模型。

**注意：设置过程最多可能需要 5 分钟。

## 导入数据

通过执行以下单元格中的内容，将导入数据并让数据可供使用。

**注意**：以下单元格中的内容代表以前的实验中的关键步骤。


In [1]:
bucket='c130335a3301602l7984601t1w760730171266-labbucket-o4qqi2tcy0re'

In [2]:
import warnings, requests, zipfile, io
warnings.simplefilter('ignore')
import pandas as pd
from scipy.io import arff

import os
import boto3
import sagemaker
from sagemaker.image_uris import retrieve
from sklearn.model_selection import train_test_split

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
f_zip = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00212/vertebral_column_data.zip'
r = requests.get(f_zip, stream=True)
Vertebral_zip = zipfile.ZipFile(io.BytesIO(r.content))
Vertebral_zip.extractall()

data = arff.loadarff('column_2C_weka.arff')
df = pd.DataFrame(data[0])

class_mapper = {b'Abnormal':1,b'Normal':0}
df['class']=df['class'].replace(class_mapper)

cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

train, test_and_validate = train_test_split(df, test_size=0.2, random_state=42, stratify=df['class'])
test, validate = train_test_split(test_and_validate, test_size=0.5, random_state=42, stratify=test_and_validate['class'])

prefix='lab3'

train_file='vertebral_train.csv'
test_file='vertebral_test.csv'
validate_file='vertebral_validate.csv'

s3_resource = boto3.Session().resource('s3')
def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(csv_buffer, header=False, index=False )
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

upload_s3_csv(train_file, 'train', train)
upload_s3_csv(test_file, 'test', test)
upload_s3_csv(validate_file, 'validate', validate)

container = retrieve('xgboost',boto3.Session().region_name,'1.0-1')

hyperparams={"num_round":"42",
             "eval_metric": "auc",
             "objective": "binary:logistic"}

s3_output_location="s3://{}/{}/output/".format(bucket,prefix)
xgb_model=sagemaker.estimator.Estimator(container,
                                       sagemaker.get_execution_role(),
                                       instance_count=1,
                                       instance_type='ml.m4.xlarge',
                                       output_path=s3_output_location,
                                        hyperparameters=hyperparams,
                                        sagemaker_session=sagemaker.Session())

train_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train/".format(bucket,prefix,train_file),
    content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validate/".format(bucket,prefix,validate_file),
    content_type='text/csv')

data_channels = {'train': train_channel, 'validation': validate_channel}

xgb_model.fit(inputs=data_channels, logs=False)

print('ready for hosting!')

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-10-16-17-05-00-009



2024-10-16 17:05:00 Starting - Starting the training job.....
2024-10-16 17:05:35 Starting - Preparing the instances for training....
2024-10-16 17:06:00 Downloading - Downloading input data.....
2024-10-16 17:06:30 Downloading - Downloading the training image.......
2024-10-16 17:07:11 Training - Training image download completed. Training in progress....
2024-10-16 17:07:31 Uploading - Uploading generated training model..
2024-10-16 17:07:44 Completed - Training job completed
ready for hosting!


# 步骤 1：托管模型

现在，您已经拥有经过训练的模型，您可以使用 Amazon SageMaker 托管服务来托管它。

第一步是部署模型。由于您有模型对象 *xgb_model*，因此可以使用 **deploy** 方法。在本实验中，将使用单个 ml.m4.xlarge 实例。



In [4]:
xgb_predictor = xgb_model.deploy(initial_instance_count=1,
                serializer = sagemaker.serializers.CSVSerializer(),
                instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-10-16-17-29-20-344
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-10-16-17-29-20-344
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-10-16-17-29-20-344


-------!

# 步骤 2：执行预测

现在，您已部署模型，您可以运行一些预测。

首先，检查测试数据并重新熟悉它。

In [5]:
test.shape

(31, 7)

您有 31 个实例，7 个属性。前五个实例是：

In [6]:
test.head(5)

,class,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
136,1,88.024499,39.844669,81.774473,48.179830,116.601538,56.766083
230,0,65.611802,23.137919,62.582179,42.473883,124.128001,-4.083298
134,1,52.204693,17.212673,78.094969,34.992020,136.972517,54.939134
130,1,50.066786,9.120340,32.168463,40.946446,99.712453,26.766697
47,1,41.352504,16.577364,30.706191,24.775141,113.266675,-4.497958


您不需要将目标值 (class) 包括在内。该预测器可以接受采用逗号分隔值 (CSV) 格式的数据。因此，您可以使用以下代码获得*不包含 class 列的*第 1 行：

`test.iloc[:1,1:]` 

**iloc** 函数接受参数 [*rows*,*cols*]

要仅获得第 1 行，请使用 `0:1`。如果要获得第 2 行，则可以使用 `1:2`。

要获得第一列 (*col 0*) *以外*的所有列，请使用 `1:`



In [7]:
row = test.iloc[0:1,1:]
row.head()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
136,88.024499,39.844669,81.774473,48.17983,116.601538,56.766083


您可以将其转换为逗号分隔值 (CSV) 文件，并将其存储在字符串缓冲区中。

In [8]:
batch_X_csv_buffer = io.StringIO()
row.to_csv(batch_X_csv_buffer, header=False, index=False)
test_row = batch_X_csv_buffer.getvalue()
print(test_row)

88.0244989,39.84466878,81.77447308,48.17983012,116.6015376,56.76608323



现在，您可以使用这些数据执行预测。

In [9]:
xgb_predictor.predict(test_row)

b'0.9966071844100952'

您得到的结果不是 *0* 或 *1*，而是*概率得分*。您可以对概率分数应用条件逻辑，以确定结果应显示为 0 还是 1。您将在进行批量预测时使用此操作。

现在，将结果与测试数据进行比较。

In [10]:
test.head(5)

,class,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
136,1,88.024499,39.844669,81.774473,48.179830,116.601538,56.766083
230,0,65.611802,23.137919,62.582179,42.473883,124.128001,-4.083298
134,1,52.204693,17.212673,78.094969,34.992020,136.972517,54.939134
130,1,50.066786,9.120340,32.168463,40.946446,99.712453,26.766697
47,1,41.352504,16.577364,30.706191,24.775141,113.266675,-4.497958


**问题**：预测准确吗？

**挑战任务**：更新之前的代码以发送数据集的第二行。这些预测正确吗？ 对其他几行尝试执行此任务。

一次发送一行可能很麻烦。您可以编写函数来批量提交这些值，但 SageMaker 本身已具备批处理功能。接下来，您将了解该功能。但在这之前，您需要终止模型。

# 步骤 3：终止已部署的模型

要删除终端节点，请在预测器上使用 **delete_endpoint** 函数。

In [11]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

INFO:sagemaker:Deleting endpoint configuration with name: sagemaker-xgboost-2024-10-16-17-29-20-344
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2024-10-16-17-29-20-344


# 步骤 4：执行批次转换

在进行训练、测试、特征工程循环期间，您希望在模型上测试保留或测试集。然后，您可以使用这些结果来计算指标。您可以像之前那样部署终端节点，但是之后必须记住删除该终端节点。但是，有一种更高效的方法。

您可以使用模型的转换器方法来获取转换器对象。然后，您可以使用此对象的转换器方法对整个测试数据集执行预测。SageMaker 将： 

- 启动具有该模型的实例
- 对所有输入值执行预测
- 将这些值写入 Amazon Simple Storage Service (Amazon S3) 
- 最后，终止实例

首先，将数据转换为 CSV 文件，以便转换器对象可以将其作为输入。这次，您将使用 **iloc** 以获取所有行以及除第一列外的所有列。


In [12]:
batch_X = test.iloc[:,1:];
batch_X.head()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
136,88.024499,39.844669,81.774473,48.179830,116.601538,56.766083
230,65.611802,23.137919,62.582179,42.473883,124.128001,-4.083298
134,52.204693,17.212673,78.094969,34.992020,136.972517,54.939134
130,50.066786,9.120340,32.168463,40.946446,99.712453,26.766697
47,41.352504,16.577364,30.706191,24.775141,113.266675,-4.497958


接下来，将数据写入 CSV 文件。

In [13]:
batch_X_file='batch-in.csv'
upload_s3_csv(batch_X_file, 'batch-in', batch_X)

最后，在执行转换之前，需要配置转换器的输入文件、输出位置和实例类型。

In [14]:
batch_output = "s3://{}/{}/batch-out/".format(bucket,prefix)
batch_input = "s3://{}/{}/batch-in/{}".format(bucket,prefix,batch_X_file)

xgb_transformer = xgb_model.transformer(instance_count=1,
                                       instance_type='ml.m4.xlarge',
                                       strategy='MultiRecord',
                                       assemble_with='Line',
                                       output_path=batch_output)

xgb_transformer.transform(data=batch_input,
                         data_type='S3Prefix',
                         content_type='text/csv',
                         split_type='Line')
xgb_transformer.wait()

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-10-16-17-37-04-736
INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-10-16-17-37-05-438


........................................[2024-10-16:17:43:49:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-16:17:43:49:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-16:17:43:49:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      return 404 "{}";
    }
  }
}
[2024-10-16 17:43:49 +0000] [19] [INFO] Star

转换完成后，您可以从 Amazon S3 下载结果并将其与输入进行比较。

首先，从 Amazon S3 下载输出并将其加载到 pandas DataFrame 中。


In [16]:
# s3 = boto3.client('s3')
# obj = s3.get_object(Bucket=bucket, Key="{}/batch-out/{}".format(prefix,'batch-in.csv.out'))
# target_predicted = pd.read_csv(io.BytesIO(obj['Body'].read()),',',names=['class'])
# target_predicted.head(5)
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key="{}/batch-out/{}".format(prefix, 'batch-in.csv.out'))
target_predicted = pd.read_csv(io.BytesIO(obj['Body'].read()), sep=',', names=['class'])
target_predicted.head(5)

,class
0,0.996607
1,0.777283
2,0.994641
3,0.993690
4,0.939139


您可以使用函数将概率转换为 *0* 或 *1*。

第一个表输出将是*预测值*，第二个表输出将是*原始测试数据*。

In [17]:
def binary_convert(x):
    threshold = 0.65
    if x > threshold:
        return 1
    else:
        return 0

target_predicted['binary'] = target_predicted['class'].apply(binary_convert)

print(target_predicted.head(10))
test.head(10)

      class  binary
0  0.996607       1
1  0.777283       1
2  0.994641       1
3  0.993690       1
4  0.939139       1
5  0.997396       1
6  0.991977       1
7  0.987518       1
8  0.993334       1
9  0.682776       1


,class,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
136,1,88.024499,39.844669,81.774473,48.179830,116.601538,56.766083
230,0,65.611802,23.137919,62.582179,42.473883,124.128001,-4.083298
134,1,52.204693,17.212673,78.094969,34.992020,136.972517,54.939134
130,1,50.066786,9.120340,32.168463,40.946446,99.712453,26.766697
47,1,41.352504,16.577364,30.706191,24.775141,113.266675,-4.497958
135,1,77.121344,30.349874,77.481083,46.771470,110.611148,82.093607
100,1,84.585607,30.361685,65.479486,54.223922,108.010218,25.118478
89,1,71.186811,23.896201,43.696665,47.290610,119.864938,27.283985
297,0,45.575482,18.759135,33.774143,26.816347,116.797007,3.131910
4,1,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501


**注意**：**binary_convert** 函数中的 *threshold* 设置为 *0.65*。

**挑战任务**：尝试更改阈值。它会影响结果吗？

**注意**：初始模型可能并不好。您将在下一个实验中生成一些指标，然后在最后一个实验中优化模型。

# 恭喜！

您已经完成了本实验，现在可以按照实验指南中的说明结束本实验。